<a href="https://colab.research.google.com/github/MiguelLuan/TelecomX_BR/blob/main/TelecomX_BR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 Extração

In [1]:
import pandas as pd

In [2]:
url = 'https://raw.githubusercontent.com/MiguelLuan/TelecomX_BR/main/Data_base/TelecomX_Data.json'

In [3]:
df = pd.read_json(url)

# 🔧 Transformação

In [4]:
customer_col = pd.json_normalize(df['customer'])
phone_col = pd.json_normalize(df['phone'])
internet_col = pd.json_normalize(df['internet'])
account_col = pd.json_normalize(df['account'],sep='_')

In [5]:
df_base = df.drop(columns=['customer','phone','internet','account'])

In [6]:
df = pd.concat([df_base,customer_col,phone_col,internet_col,account_col],axis=1)

In [7]:
df

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,9987-LUTYD,No,Female,0,No,No,13,Yes,No,DSL,...,No,No,Yes,No,No,One year,No,Mailed check,55.15,742.9
7263,9992-RRAMN,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber optic,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.7
7264,9992-UJOEL,No,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75
7265,9993-LHIEB,No,Male,0,Yes,Yes,67,Yes,No,DSL,...,No,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


In [9]:
for i in df.columns:
  print(i,'-',df[i].unique())

customerID - ['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
Churn - ['No' 'Yes' '']
gender - ['Female' 'Male']
SeniorCitizen - [0 1]
Partner - ['Yes' 'No']
Dependents - ['Yes' 'No']
tenure - [ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 11 37 49 66
 67 20 43 59 12 27  2 25 29 14 35 64 39 40  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 51 31 36 17 18 38 42
  0]
PhoneService - ['Yes' 'No']
MultipleLines - ['No' 'Yes' 'No phone service']
InternetService - ['DSL' 'Fiber optic' 'No']
OnlineSecurity - ['No' 'Yes' 'No internet service']
OnlineBackup - ['Yes' 'No' 'No internet service']
DeviceProtection - ['No' 'Yes' 'No internet service']
TechSupport - ['Yes' 'No' 'No internet service']
StreamingTV - ['Yes' 'No' 'No internet service']
StreamingMovies - ['No' 'Yes' 'No internet service']
Contract - ['One year' 'Month-to-month' 'Two year']
PaperlessBilling - ['Yes' 'No']
PaymentMethod - ['Mailed check' 'El

In [10]:
sum(df['Charges_Total'].str.match(r'^\s*$'))

11

In [11]:
df['Charges_Total'] = pd.to_numeric(df['Charges_Total'],errors='coerce')


In [12]:
df['Charges_Total'].isna().sum()

np.int64(11)

In [13]:
df = df.dropna(subset=['Charges_Total'])

In [14]:
len(df.index)

7256

In [15]:
df = df[df['Churn'] != '' ]

In [16]:
cols_num = ['Churn','Partner','Dependents','PhoneService','PaperlessBilling']

for col in cols_num:
  df[col] = (df[col]
          .str.replace('Yes','1')
          .str.replace('No','0')
          .astype('Int64')
  )

In [17]:
df = df.astype(
    {
      'customerID': 'string',
      'gender': 'category',
      'MultipleLines': 'category',
      'InternetService': 'category',
      'OnlineSecurity': 'category',
      'OnlineBackup': 'category',
      'DeviceProtection': 'category',
      'TechSupport': 'category',
      'StreamingTV': 'category',
      'StreamingMovies': 'category',
      'Contract': 'category',
      'PaymentMethod': 'category',
    }
)

In [18]:
df['customerID'].duplicated().sum()

np.int64(0)

In [19]:
df = df.assign( daily_accounts = round(df['Charges_Monthly'] / 30,2))

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7266
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   customerID        7032 non-null   string  
 1   Churn             7032 non-null   Int64   
 2   gender            7032 non-null   category
 3   SeniorCitizen     7032 non-null   int64   
 4   Partner           7032 non-null   Int64   
 5   Dependents        7032 non-null   Int64   
 6   tenure            7032 non-null   int64   
 7   PhoneService      7032 non-null   Int64   
 8   MultipleLines     7032 non-null   category
 9   InternetService   7032 non-null   category
 10  OnlineSecurity    7032 non-null   category
 11  OnlineBackup      7032 non-null   category
 12  DeviceProtection  7032 non-null   category
 13  TechSupport       7032 non-null   category
 14  StreamingTV       7032 non-null   category
 15  StreamingMovies   7032 non-null   category
 16  Contract          7032 non-nu

# 📊 Carga e análise

In [21]:
total_cancelados = sum(df['Churn'] == 1)


In [22]:
df[df['Churn'] == 1].agg(
    {
        'customerID':'size',
        'tenure':'mean',
        'daily_accounts':'mean',
        'Charges_Monthly':'mean',
        'Charges_Total':'mean',
    }
).apply(lambda x: round(x,2)).rename('Metricas')

,Metricas
customerID,1869.00
tenure,17.98
daily_accounts,2.48
Charges_Monthly,74.44
Charges_Total,1531.80


In [23]:
df[df['Churn'] == 0].agg(
    {
        'customerID': 'size',
        'tenure':'mean',
        'daily_accounts':'mean',
        'Charges_Monthly':'mean',
        'Charges_Total':'mean',
    }
).apply(lambda x: round(x,2)).rename('Metricas')

,Metricas
customerID,5163.00
tenure,37.65
daily_accounts,2.04
Charges_Monthly,61.31
Charges_Total,2555.34


In [24]:
perc_cancelamento = round(sum(df['Churn'] == 1) / len(df) * 100,2)
print(f'O número de cancelamento corresponde a {perc_cancelamento}% do total {len(df)}')

O número de cancelamento corresponde a 26.58% do total 7032


In [25]:
pd.crosstab(df['gender'], df['Churn'], normalize='columns').apply(lambda x: round(x,4)*100)


Churn,0,1
gender,,
Female,49.27,50.24
Male,50.73,49.76


A taxa de churn é praticamente igual entre os gêneros, logo gênero não é fator relevante.

In [26]:
pd.crosstab(df['SeniorCitizen'],df['Churn'],normalize='columns').apply(lambda x: round(x*100,2))

Churn,0,1
SeniorCitizen,,
0,87.1,74.53
1,12.9,25.47


In [27]:
pd.crosstab(df['Partner'],df['Churn'],normalize='columns').apply(lambda x: round(x*100,2))

Churn,0,1
Partner,,
0,47.24,64.21
1,52.76,35.79


Clientes sem parceiro apresentam uma taxa de churn significativamente maior do que clientes com parceiro, indicando que a presença de um parceiro está associada a maior retenção.

In [28]:
pd.crosstab(df['Dependents'],df['Churn'],normalize='columns').apply(lambda x: round(x*100,2))

Churn,0,1
Dependents,,
0,65.66,82.56
1,34.34,17.44


Clientes que não possui dependentes tende a ter uma taxa de churn significativamente elevada em comparação daqueles que possuem

In [29]:
for i in ['PhoneService','InternetService','MultipleLines','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']:
  print(pd.crosstab(df[i],df['Churn'],normalize='columns').apply(lambda x: round(x*100,2)))
  print('-'*30,'\n')

Churn             0     1
PhoneService             
0              9.88   9.1
1             90.12  90.9
------------------------------ 

Churn                0      1
InternetService              
DSL              37.90  24.56
Fiber optic      34.84  69.40
No               27.25   6.05
------------------------------ 

Churn                 0      1
MultipleLines                 
No                49.12  45.43
No phone service   9.88   9.10
Yes               41.00  45.48
------------------------------ 

Churn                    0      1
OnlineSecurity                   
No                   39.43  78.17
No internet service  27.25   6.05
Yes                  33.31  15.78
------------------------------ 

Churn                    0      1
OnlineBackup                     
No                   35.91  65.97
No internet service  27.25   6.05
Yes                  36.84  27.98
------------------------------ 

Churn                    0      1
DeviceProtection                 
No                

In [31]:
for i in ['Contract','PaymentMethod','PaperlessBilling']:
  print(pd.crosstab(df[i],df['Churn'],normalize='columns').apply(lambda x: round(x*100,2)))
  print('-'*30,'\n')

Churn               0      1
Contract                    
Month-to-month  43.00  88.55
One year        25.30   8.88
Two year        31.71   2.57
------------------------------ 

Churn                          0      1
PaymentMethod                          
Bank transfer (automatic)  24.87  13.80
Credit card (automatic)    24.97  12.41
Electronic check           25.06  57.30
Mailed check               25.10  16.48
------------------------------ 

Churn                 0      1
PaperlessBilling              
0                 46.39  25.09
1                 53.61  74.91
------------------------------ 

